In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 4.2 MB/s 
     |████████████████████████████████| 1.6 MB 43.8 MB/s 
     |████████████████████████████████| 9.3 MB 34.2 MB/s 
     |████████████████████████████████| 108 kB 53.9 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,1,0,5,0,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,1,2,3,0,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,6.115208,7.919091,271616,7.886012,0,1


In [6]:
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
685,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,6.519312,5.299702,485200,6.442827,1,0
3817,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,40448,7.618580,1,1
4525,80,2,4,65535,184,256,6,1,0,1,...,1,4,2,0,0.000000,0.000000,90112,7.564760,0,1
2746,144,3,4,65535,184,224,7,1,0,1,...,1,4,3,0,6.036959,1.454411,149504,7.587887,1,1
5134,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,407040,6.432705,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4201,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.319689,0.000000,95232,7.602991,1,1
3257,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,5.999184,1.602594,380928,6.890686,0,1
959,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.360474,0.393634,105432,6.021354,1,0
1620,80,2,4,65535,184,256,8,1,1,1,...,1,4,4,0,0.000000,0.000000,1128916,7.984075,0,0


In [8]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [9]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [10]:
df

,index,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,685,144,3,4,65535,184,264,5,1,0,...,1,1,4,0,6.519312,5.299702,485200,6.442827,1,0
1,3817,144,3,4,65535,184,192,3,1,1,...,1,2,1,1,0.000000,0.000000,40448,7.618580,1,1
2,4525,80,2,4,65535,184,256,6,1,0,...,1,4,2,0,0.000000,0.000000,90112,7.564760,0,1
3,2746,144,3,4,65535,184,224,7,1,0,...,1,4,3,0,6.036959,1.454411,149504,7.587887,1,1
4,5134,80,2,4,65535,184,256,8,1,0,...,1,4,4,1,0.000000,0.000000,407040,6.432705,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,4201,144,3,4,65535,184,232,6,1,0,...,1,2,4,0,6.319689,0.000000,95232,7.602991,1,1
5206,3257,144,3,4,65535,184,232,4,1,1,...,1,0,4,1,5.999184,1.602594,380928,6.890686,0,1
5207,959,144,3,4,65535,184,240,5,1,0,...,1,1,4,0,6.360474,0.393634,105432,6.021354,1,0
5208,1620,80,2,4,65535,184,256,8,1,1,...,1,4,4,0,0.000000,0.000000,1128916,7.984075,0,0


In [11]:
df.drop(['index'],axis=1,inplace=True)

In [12]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,6.519312,5.299702,485200,6.442827,1,0
1,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,40448,7.618580,1,1
2,80,2,4,65535,184,256,6,1,0,1,...,1,4,2,0,0.000000,0.000000,90112,7.564760,0,1
3,144,3,4,65535,184,224,7,1,0,1,...,1,4,3,0,6.036959,1.454411,149504,7.587887,1,1
4,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,407040,6.432705,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.319689,0.000000,95232,7.602991,1,1
5206,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,5.999184,1.602594,380928,6.890686,0,1
5207,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.360474,0.393634,105432,6.021354,1,0
5208,80,2,4,65535,184,256,8,1,1,1,...,1,4,4,0,0.000000,0.000000,1128916,7.984075,0,0


In [13]:
df_5percent=df[0:260] 

In [14]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,6.519312,5.299702,485200,6.442827,1,0
1,144,3,4,65535,184,192,3,1,1,1,...,1,2,1,1,0.000000,0.000000,40448,7.618580,1,1
2,80,2,4,65535,184,256,6,1,0,1,...,1,4,2,0,0.000000,0.000000,90112,7.564760,0,1
3,144,3,4,65535,184,224,7,1,0,1,...,1,4,3,0,6.036959,1.454411,149504,7.587887,1,1
4,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,1,0.000000,0.000000,407040,6.432705,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,144,3,4,65535,184,248,5,1,1,1,...,1,2,3,0,0.000000,0.000000,123904,5.413136,0,1
256,80,2,4,65535,184,256,8,1,0,1,...,1,4,4,0,0.000000,0.000000,28160,5.881307,0,1
257,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,0,5.914972,0.000000,250784,5.944966,0,0
258,144,3,4,65535,184,248,5,1,1,1,...,1,2,3,0,0.000000,6.516784,114176,5.578488,0,1


In [15]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (129, 69)
class 1: (131, 69)


In [16]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [17]:
df_5percent.info

<bound method DataFrame.info of      e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0     144.0   3.0        4.0     65535.0  184.0     264.0               5.0   
1     144.0   3.0        4.0     65535.0  184.0     192.0               3.0   
2      80.0   2.0        4.0     65535.0  184.0     256.0               6.0   
3     144.0   3.0        4.0     65535.0  184.0     224.0               7.0   
4      80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
..      ...   ...        ...         ...    ...       ...               ...   
255   144.0   3.0        4.0     65535.0  184.0     248.0               5.0   
256    80.0   2.0        4.0     65535.0  184.0     256.0               8.0   
257   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
258   144.0   3.0        4.0     65535.0  184.0     248.0               5.0   
259   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   

     CreationYear  

In [18]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,260.000000,260.000000,260.000000,260.0,260.000000,260.000000,260.000000,260.000000,260.000000,260.0,...,260.0,260.000000,260.000000,260.000000,260.000000,260.000000,2.600000e+02,260.000000,260.000000,260.000000
mean,139.323077,2.930769,3.969231,65535.0,182.584615,221.169231,4.615385,0.988462,0.346154,1.0,...,1.0,1.373077,3.242308,0.173077,4.875605,2.582887,7.991354e+05,6.308396,0.553846,0.503846
std,16.688939,0.254336,0.350145,0.0,16.106683,41.131325,1.734878,0.107002,0.476660,0.0,...,0.0,1.520531,1.075768,0.379043,2.605821,2.698419,6.794187e+06,1.154065,0.498051,0.500949
min,80.000000,2.000000,0.000000,65535.0,0.000000,124.000000,1.000000,0.000000,0.000000,1.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,4.608000e+03,0.967567,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.0,184.000000,200.000000,4.000000,1.000000,0.000000,1.0,...,1.0,1.000000,3.000000,0.000000,3.638975,0.000000,6.022600e+04,5.684796,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.0,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.0,1.000000,3.000000,0.000000,6.197033,1.695914,1.249280e+05,6.374232,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.0,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.0,2.000000,4.000000,0.000000,6.489545,4.642553,3.049000e+05,7.185474,1.000000,1.000000
max,144.000000,3.000000,4.000000,65535.0,184.000000,320.000000,17.000000,1.000000,1.000000,1.0,...,1.0,14.000000,7.000000,1.000000,7.999131,7.999291,1.089712e+08,7.999997,1.000000,1.000000


In [19]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [20]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [21]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [22]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      53.75
1.0      46.25
dtype: float64

In [23]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      57.5
0.0      42.5
dtype: float64

In [24]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))